# **RA1: quarterly panel data**

Dong Gyun Ko <br/>
last updated: september 5, 2022 <br/>

In [38]:
# import libraries

import numpy as np
import pandas as pd
import datetime as dt
import pandas_datareader as pdr
import pandas_datareader.data as web
from pandas_datareader import wb
from pandas_datareader.data import DataReader
import requests # python 3.6

import os
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [39]:
# set the working directory

os.chdir('/content/gdrive/MyDrive/Colab Notebooks/RA1_data')

In [40]:
# set the pandas display option

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

# pd.options.display.max_rows = 100
# pd.options.display.max_columns = 100

## **1. panel data**

### **1.1. python code** <br/>

In [41]:
# import the pooled data

df_q_pooled = pd.read_excel('df_q_pooled.xlsx').drop(columns=['Unnamed: 0'])
df_q_pooled['quarter'] = pd.to_datetime(df_q_pooled['quarter'])
df_q_pooled['quarter'] = df_q_pooled['quarter'].dt.to_period('Q')

In [42]:
# quarterly pooled dataset

df_q_pooled = df_q_pooled.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)

df_q_pooled

,country,isocode,quarter,oecd_q_ggexp,oecd_q_gginv,oecd_q_rgdp_dcu,oecd_q_rgdp_usd,oecd_q_ca,oecd_q_stir,bis_q_ggdebt_m,...,imf_q_gfcf,imf_q_cpi,imf_q_gdpd,imf_q_neer,imf_q_reer,imf_q_ca,imf_q_stir_mm,imf_q_stir_3mtb,imf_q_stir_mp,fred_q_ggexp
0,Aruba,ABW,1913Q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aruba,ABW,1913Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Aruba,ABW,1913Q3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aruba,ABW,1913Q4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aruba,ABW,1914Q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95041,Zimbabwe,ZWE,2021Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4984.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95042,Zimbabwe,ZWE,2021Q3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5520.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95043,Zimbabwe,ZWE,2021Q4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,6497.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95044,Zimbabwe,ZWE,2022Q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
# generate the count matrix

count_N_df_q_pooled = df_q_pooled.groupby(df_q_pooled['isocode']).count()
count_T_df_q_pooled = df_q_pooled.groupby(df_q_pooled['quarter']).count()

In [44]:
# filter (N)

# ggdebt
condN1 = (count_N_df_q_pooled['bis_q_ggdebt_m'] == 0)
condN2 = (count_N_df_q_pooled['bis_q_ggdebt_n'] == 0)
condN3 = (count_N_df_q_pooled['wb_q_ggdebt_d2'] == 0)
condN4 = (count_N_df_q_pooled['eustat_q_ggdebt'] == 0)
condN5 = (count_N_df_q_pooled['imf_q_ggdebt'] == 0)

# ggexp
condN6 = (count_N_df_q_pooled['oecd_q_ggexp'] == 0)
condN7 = (count_N_df_q_pooled['eustat_q_ggexp'] == 0)
condN8 = (count_N_df_q_pooled['imf_q_ggexp'] == 0)
condN9 = (count_N_df_q_pooled['fred_q_ggexp'] == 0)

count_N_df_q_pooled = count_N_df_q_pooled.loc[(~condN1 | ~condN2 | ~condN3 | ~condN4 | ~condN5) & (~condN6 | ~condN7 | ~condN8 | ~condN9)]
count_N_df_q_pooled = count_N_df_q_pooled.reset_index()

print('countries:', count_N_df_q_pooled['isocode'].unique().shape[0])

countries: 50


In [45]:
# filter (T)

# ggdebt
condT1 = (count_T_df_q_pooled['bis_q_ggdebt_m'] == 0)
condT2 = (count_T_df_q_pooled['bis_q_ggdebt_n'] == 0)
condT3 = (count_T_df_q_pooled['wb_q_ggdebt_d2'] == 0)
condT4 = (count_T_df_q_pooled['eustat_q_ggdebt'] == 0)
condT5 = (count_N_df_q_pooled['imf_q_ggdebt'] == 0)

# ggexp
condT6 = (count_T_df_q_pooled['oecd_q_ggexp'] == 0)
condT7 = (count_T_df_q_pooled['eustat_q_ggexp'] == 0)
condT8 = (count_T_df_q_pooled['imf_q_ggexp'] == 0)
condT9 = (count_T_df_q_pooled['fred_q_ggexp'] == 0)

count_T_df_q_pooled = count_T_df_q_pooled.loc[(~condT1 | ~condT2 | ~condT3 | ~condT4 | ~condT5) & (~condT6 | ~condT7 | ~condT8 | ~condT9)]
count_T_df_q_pooled = count_T_df_q_pooled.reset_index()

print('start:', count_T_df_q_pooled['quarter'].unique().min())

start: 1955Q1


/usr/local/lib/python3.7/dist-packages/pandas/core/indexes/base.py:4024: RuntimeWarning: '<' not supported between instances of 'int' and 'Period', sort order is undefined for incomparable objects
  join_index = self.union(other)


In [46]:
# generate the panel data

# select the T
cond_T = (df_q_pooled['quarter'].dt.year >= 1955)

df_q_panel = df_q_pooled.loc[cond_T]

# select the N
print(count_N_df_q_pooled['isocode'].unique().shape[0])
isocode_list = count_N_df_q_pooled['isocode'].unique()

for i in range(0, count_N_df_q_pooled['isocode'].unique().shape[0]):
  
    globals()['cond_N{}'.format(i+1)] = (df_q_pooled['isocode'] == isocode_list[i])

df_q_panel = df_q_panel.loc[cond_N1 | cond_N2 | cond_N3 | cond_N4 | cond_N5 | cond_N6 | cond_N7 | cond_N8 | cond_N9 | cond_N10 | cond_N11 | cond_N12 | cond_N13 | cond_N14 | cond_N15 | cond_N16 | cond_N17 | cond_N18 | cond_N19 | cond_N20 | cond_N21 | cond_N22 | cond_N23 | cond_N24 | cond_N25 | cond_N26 | cond_N27 | cond_N28 | cond_N29 | cond_N30 | cond_N31 | cond_N32 | cond_N33 | cond_N34 | cond_N35 | cond_N36 | cond_N37 | cond_N38 | cond_N39 | cond_N40 | cond_N41 | cond_N42 | cond_N43 | cond_N44 | cond_N45 | cond_N46 | cond_N47 | cond_N48 | cond_N49 | cond_N50]

50


In [47]:
# quarterly panel data

df_q_panel = df_q_panel.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)

df_q_panel

,country,isocode,quarter,oecd_q_ggexp,oecd_q_gginv,oecd_q_rgdp_dcu,oecd_q_rgdp_usd,oecd_q_ca,oecd_q_stir,bis_q_ggdebt_m,...,imf_q_gfcf,imf_q_cpi,imf_q_gdpd,imf_q_neer,imf_q_reer,imf_q_ca,imf_q_stir_mm,imf_q_stir_3mtb,imf_q_stir_mp,fred_q_ggexp
0,Argentina,ARG,1955Q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Argentina,ARG,1955Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Argentina,ARG,1955Q3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Argentina,ARG,1955Q4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Argentina,ARG,1956Q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13495,South Africa,ZAF,2021Q2,NaN,NaN,NaN,NaN,NaN,8.60,NaN,...,158173000000.00,170.41,178.79,61.51,80.20,7139185452.00,3.51,3.72,3.50,222850.40
13496,South Africa,ZAF,2021Q3,NaN,NaN,NaN,NaN,NaN,9.16,NaN,...,156396000000.00,173.17,181.62,60.05,78.67,2482808479.00,3.53,3.81,3.50,224008.70
13497,South Africa,ZAF,2021Q4,NaN,NaN,NaN,NaN,NaN,10.16,NaN,...,158836000000.00,174.99,183.21,57.79,75.72,3494471541.00,3.63,3.81,3.75,224479.80
13498,South Africa,ZAF,2022Q1,NaN,NaN,NaN,NaN,NaN,10.26,NaN,...,164619000000.00,177.63,184.89,58.99,76.99,623774268.60,3.96,4.01,4.25,226828.70


### **1.2. var. summary** <br/>

| |variable|abbreviation|unit|frequency|indicator|subject|measure|source|
|-|--------|------------|----|---------|---------|-------|-------|------|
|$OECD$|||||||||
|1|government final consumption expenditure, volume|oecd_q_ggexp|DCU, base year|quarterly|-|-|-|https://stats.oecd.org/Index.aspx?QueryId=61357#|
|2|goverment gross fixed capital formation, volume|oecd_q_gginv|DCU, base year|quarterly|-|-|-|https://stats.oecd.org/Index.aspx?QueryId=61357#|
|3|gross domestic product, volume, market prices|oecd_q_rgdp_dcu|DCU, base year|quarterly|-|-|-|https://stats.oecd.org/Index.aspx?QueryId=61357#|
|4|gross domestic product, volume in USD, constant exchange rates|oecd_q_rgdp_usd|USD, 2015|quarterly|-|-|-|https://stats.oecd.org/Index.aspx?QueryId=61357#|
|5|current account balance as a percentage of GDP|oecd_q_ca|% of GDP|quarterly|-|-|-|https://stats.oecd.org/Index.aspx?QueryId=61357#|
|6|short-term interest rate|oecd_q_stir|% per annum|quarterly|-|-|-|https://stats.oecd.org/Index.aspx?QueryId=61357#|
|$BIS$|||||||||
|1|total credit to the government sector at market value (core debt)|bis_q_ggdebt_m|% of GDP|quarterly|CRE|-|market value|https://stats.bis.org/statx/srs/table/f5.1|
|2|total credit to the government sector at nominal value (core debt)|bis_q_ggdebt_n|% of GDP|quarterly|CRE|-|nominal value|https://stats.bis.org/statx/srs/table/f5.4|
|3|consumer price index - BIS spliced, not seasonally adjusted|bis_q_cpi|2010=100|quarterly|CPI|-|-|https://stats.bis.org/statx/srs/table/k1|
|4|central bank policy rates|bis_q_cbpr|%|quarterly|CBP|-|-|https://stats.bis.org/statx/srs/table/l1|
|5|real effective exchange rate, broad(60 economies) indicies|bis_q_reer_b|2010=100|quarterly|EER|-|-|https://stats.bis.org/statx/srs/table/i2?m=B|
|6|real effective exchange rate, narrow(27 economies) indicies|bis_q_reer_n|2010=100|quarterly|EER|-|-|https://stats.bis.org/statx/srs/table/i2?m=B|
|$World Bank$|||||||||
|1|Gross PSD, General Gov.-D1, All maturities, Debt securities + loans, Nominal Value, % of GDP|wb_q_ggdebt_d1|% of GDP|quarterly|DP.DOD.DLD1.CR.GG.Z1|-|nominal value|https://databank.worldbank.org/source/Quarterly-Public-Sector-Debt/#|
|2|Gross PSD, General Gov.-D2, All maturities, D1+ SDRs + currency and deposits, Nominal Value, % of GDP|wb_q_ggdebt_d2|% of GDP|quarterly|DP.DOD.DLD2.CR.GG.Z1|-|nominal value|https://databank.worldbank.org/source/Quarterly-Public-Sector-Debt/#|
|3|Gross PSD, General Gov.-D3, All maturities, D2+other accounts payable, Nominal Value, % of GDP|wb_q_ggdebt_d3|% of GDP|quarterly|DP.DOD.DLD3.CR.GG.Z1|-|nominal value|https://databank.worldbank.org/source/Quarterly-Public-Sector-Debt/#|
|4|Gross PSD, General Gov.-D4, All maturities, D3+insurance, pensions, and standardized guarantees, Nominal Value, % of GDP|wb_q_ggdebt_d4|% of GDP|quarterly|DP.DOD.DLD2.CR.GG.Z1|-|nominal value|https://databank.worldbank.org/source/Quarterly-Public-Sector-Debt/#|
|$Eurostat$|||||||||
|1|government consolidated gross debt [S13: general government]|eustat_q_ggdebt|% of GDP|quarterly|GOV_10Q_GGDEBT [GD]|-|PC_GDP|https://ec.europa.eu/eurostat/databrowser/view/GOV_10Q_GGDEBT__custom_3260679/default/table?lang=en|
|2|final consumption expenditure [S13: general government]|eustat_q_ggexp|% of GDP|quarterly|GOV_10Q_GGNFA [P3]|-|PC_GDP / NSA|https://ec.europa.eu/eurostat/databrowser/view/GOV_10Q_GGNFA__custom_3263185/default/table?lang=en|
|3|gross fixed capital formation [S13: general government]|eustat_q_gginv|% of GDP|quarterly|GOV_10Q_GGNFA [P51G]|-|PC_GDP / NSA|https://ec.europa.eu/eurostat/databrowser/view/GOV_10Q_GGNFA__custom_3263870/default/table?lang=en|
|$IMF$|||||||||
|1|fiscal, general government, assets and liabilities, debt|imf_q_ggdebt|DCU|quarterly|GG_GALM_G01_XDC|-|market val.|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|
|2|general government final consumption expenditure, real|imf_q_ggexp|DCU|quarterly|NCGG_R_SA_XDC|-|real / SA|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|
|3|gross domestic product, nominal|imf_q_ngdp|DCU|quarterly|NGDP_SA_XDC|-|nominal / SA|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|
|4|gross domestic product, real|imf_q_rgdp|DCU|quarterly|NGDP_R_SA_XDC|-|real / SA|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|
|5|final consumption expenditure, real|imf_q_exp|DCU|quarterly|NC_R_SA_XDC|-|real / SA|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|
|6|private sector final consumption expenditure, real|imf_q_pexp|DCU|quarterly|NCP_R_SA_XDC|-|real, SA|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|
|7|gross capital formation, real|imf_q_gcf|DCU|quarterly|NI_R_SA_XDC|-|real / SA|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|
|8|gross fixed capital formation, real|imf_q_gfcf|DCU|quarterly|NFI_R_SA_XDC|-|real / SA|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|
|9|prices, consumer price index, all items|imf_q_cpi|index|quarterly|PCPI_IX|-|-|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|
|10|gross domestic product, deflator|imf_q_gdpd|index|quarterly|NGDP_D_SA_IX|-|SA|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|
|11|exchange rates, nominal effective exchange rate|imf_q_neer|index|quarterly|ENEER_IX|-|nominal|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|
|12|exchange rates, real effective exchange rate based on consumer price index|imf_q_reer|index|quarterly|EREER_IX|-|real|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|
|13|supplementary items, current account, net (excluding exceptional financing)|imf_q_ca|USD|quarterly|BCAXF_BP6_USD|-|-|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|
|14|financial, interest rates, money market|imf_q_stir_mm|percent per annum|quarterly|FIMM_PA|-|-|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|
|15|financial, interest rates, government securities, treasury bills, 3-month|imf_q_stir_3mtb|percent per annum|quarterly|FITB_3M_PA|-|-|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|
|16|financial, interest rates, monetary policy-related interest rate|imf_q_stir_mp|percent per annum|quarterly|FPOLM_PA|-|-|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|
|$FRED$|||||||||
|1|real general government final consumption expenditure|fred_q_ggexp|DCU|quarterly|NCGGRSAXDC + isocode [alpha-2] + Q|-|real / SA|https://fred.stlouisfed.org/searchresults?st=real+general+government+final+consumption+expenditure|

### **1.3. data summary**

In [48]:
# number of countries & sample period

print('number of countries:', df_q_panel['isocode'].unique().shape[0])
print(df_q_panel['isocode'].unique())
print('start:', np.min(df_q_panel['quarter'].unique()))
print('end:', np.max(df_q_panel['quarter'].unique()))

number of countries: 50
['ARG' 'AUS' 'AUT' 'BEL' 'BGR' 'BRA' 'CAN' 'CHE' 'CHL' 'COL' 'CYP' 'CZE'
 'DEU' 'DNK' 'ESP' 'EST' 'FIN' 'FRA' 'GBR' 'GRC' 'HKG' 'HRV' 'HUN' 'IDN'
 'IND' 'IRL' 'ISR' 'ITA' 'JPN' 'KOR' 'LTU' 'LUX' 'LVA' 'MEX' 'MLT' 'NLD'
 'NOR' 'NZL' 'POL' 'PRT' 'ROU' 'RUS' 'SGP' 'SVK' 'SVN' 'SWE' 'THA' 'TUR'
 'USA' 'ZAF']
start: 1955Q1
end: 2022Q2


In [49]:
# non-null count by each variables

df_q_panel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13500 entries, 0 to 13499
Data columns (total 39 columns):
 #   Column           Non-Null Count  Dtype        
---  ------           --------------  -----        
 0   country          13500 non-null  object       
 1   isocode          13500 non-null  object       
 2   quarter          13500 non-null  period[Q-DEC]
 3   oecd_q_ggexp     4340 non-null   float64      
 4   oecd_q_gginv     2574 non-null   float64      
 5   oecd_q_rgdp_dcu  4456 non-null   float64      
 6   oecd_q_rgdp_usd  4456 non-null   float64      
 7   oecd_q_ca        3534 non-null   float64      
 8   oecd_q_stir      4180 non-null   float64      
 9   bis_q_ggdebt_m   3164 non-null   float64      
 10  bis_q_ggdebt_n   4394 non-null   float64      
 11  bis_q_cpi        10776 non-null  float64      
 12  bis_q_cbpr       4583 non-null   object       
 13  bis_q_reer_b     5700 non-null   float64      
 14  bis_q_reer_n     5850 non-null   float64      
 15  wb

In [50]:
# descriptive statistics

pd.options.display.float_format = '{:.2f}'.format
round(df_q_panel.describe(), 2)

,oecd_q_ggexp,oecd_q_gginv,oecd_q_rgdp_dcu,oecd_q_rgdp_usd,oecd_q_ca,oecd_q_stir,bis_q_ggdebt_m,bis_q_ggdebt_n,bis_q_cpi,bis_q_reer_b,...,imf_q_gfcf,imf_q_cpi,imf_q_gdpd,imf_q_neer,imf_q_reer,imf_q_ca,imf_q_stir_mm,imf_q_stir_3mtb,imf_q_stir_mp,fred_q_ggexp
count,4340.00,2574.00,4456.00,4456.00,3534.00,4180.00,3164.00,4394.00,10776.00,5700.00,...,6164.00,11206.00,6048.00,6822.00,6439.00,7445.00,5644.00,4211.00,3465.00,2406.00
mean,6542472575848.33,4110268804224.70,45445265002657.84,927588342684.50,-0.63,8.65,65.37,60.35,62.10,96.15,...,12109842952381.61,58.71,91.55,9123210976.06,96.94,-811218270.65,35.02,7.89,7.61,9786714.16
std,21341904505376.79,12179893287988.37,147422616429373.91,1994209164892.58,4.30,6.34,39.61,34.97,59.15,18.08,...,75405299633655.34,43.26,106.24,209073031794.54,16.13,18442322185.77,894.91,12.90,12.60,32293790.33
min,1365709967.41,-44992000000.00,8075649110.82,8956691869.00,-20.14,0.05,4.60,3.60,0.00,41.04,...,148800891.20,0.00,0.86,18.67,32.42,-255473000000.00,-0.68,-1.01,-0.75,9907.00
25%,36126826394.71,13771695535.06,187550515950.61,126901687722.33,-2.93,4.40,38.60,35.00,18.44,89.56,...,11624950000.00,16.29,68.41,91.36,90.02,-1747000000.00,2.67,2.59,1.75,53192.70
50%,158832000000.00,42408000000.00,760965321672.18,268975028885.86,-0.63,7.00,57.10,53.20,63.57,97.58,...,48920800000.00,61.03,91.39,99.56,97.97,-206755175.70,5.50,5.06,5.00,83369.98
75%,460097500000.00,109569253028.50,2316546626302.49,828358210890.16,1.71,11.46,84.10,75.80,97.70,102.43,...,176968500000.00,96.54,106.60,107.57,103.71,1323288969.00,10.53,8.60,8.50,415719.32
max,180434000000000.00,73184000000000.00,1332547200000000.00,16533342932403.30,18.93,57.84,238.20,226.90,2031.98,281.92,...,911382000000000.00,525.87,3553.02,8473310000000.00,203.52,94237700000.00,61846.25,278.37,350.53,230259300.00


## **2. export the data**

In [51]:
# quarterly panel data

df_q_panel.to_excel(excel_writer='df_q_panel.xlsx')

## **3. references**

* https://data.oecd.org/ <br/>
* https://stats.bis.org/#ppq=XRU_D_24D;pv=1,2~4~1,0,0~both <br/>
* https://databank.worldbank.org/source/world-development-indicators/preview/on <br/>
* https://wits.worldbank.org/wits/wits/witshelp/content/codes/country_codes.htm <br/>
* https://ec.europa.eu/eurostat/en/web/main/data/database <br/>
* https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1409151240976 <br/>
* https://fred.stlouisfed.org/ <br/>